In [ ]:
# from datasets import Dataset, DatasetDict, ClassLabel
# import numpy as np

# # Define your data dictionary
# data_dict = {
#     "image": [np.random.random((128, 128, 3)), np.random.random((128, 128, 3)), np.random.random((128, 128, 3))],
#     "label": [0, 1, 2]  # Assuming label indices correspond to class labels
# }

# # Define the ClassLabel mapping
# class_label = ClassLabel(num_classes=3, names=["Cat", "Dog", "Bird"])

# Create a Dataset using DatasetDict
# dataset = DatasetDict({
#     "train": Dataset.from_dict(data_dict, features={"image": "image", "label": class_label})
# })

# # Access and print the dataset
# print(dataset)


In [ ]:
# import tifffile
# import os
# from PIL import Image
# import numpy as np

# # Define the ClassLabel mapping
# from datasets import ClassLabel

# class_label = ClassLabel(num_classes=2, names=["nonprogressor", "progressor"])

# # Function to create PIL image
# def create_pil_image(new_image, new_image_path):
#     # Convert the data type to uint8
#     new_image = new_image.astype(np.uint8)
#     print("new_image shape:", new_image.shape)
#     print("new_image dtype:", new_image.dtype)

#     print("Min value:", np.min(new_image))
#     print("Max value:", np.max(new_image))

#     # print("new_image content:", new_image[:3, :3, :3])  # Print the first 3x3x3 subarray

#     new_image_pil = Image.fromarray(new_image, mode="RGB")
#     print("new_image_pil dtype: {} ".format(type(new_image_pil)))
#     # new_image_pil = Image.fromarray(new_image)
#     new_image_pil.save(new_image_path)
#     return(new_image_pil)

# # call this for each one of the labels
# def load_n_channel_tif_to_dataset_object(list_of_images_files, label):
#     # Define dimensions
#     original_channels = 47
#     original_rows = 1024
#     new_channels = 3
#     new_rows = 4053
#     new_cols = 4053
#     data_list = list()

#     # Initialize new_image array outside the loop
#     new_image = np.zeros((new_channels, new_rows, new_cols), dtype=np.uint8)
    
#     for image_path in list_of_images_files:
#         # Read the image
#         original_image = tifffile.imread(image_path)
        
#         # Iterate through original image channels and rows
#         for channel in range(original_channels):
#             for row in range(original_rows):
#                 new_row = (channel * original_rows + row) % new_rows
#                 if new_row < new_rows:
#                     new_image[channel % new_channels, new_row, :original_image.shape[2]] = original_image[channel, row, :]

#         # Create PIL image and save it
#         new_image_filename = os.path.basename(image_path).replace('.tif', '.jpg')
#         new_image_save_path = os.path.join(os.path.dirname(image_path), new_image_filename)
#         new_image_pil = create_pil_image(new_image, new_image_save_path)

#         # Append paths and labels to data_dict
#         data_item = {
#             'image': new_image_pil,
#             'image_file_path': new_image_save_path,
#             'labels': class_label.str2int(label)
#         }
#         data_list.append(data_item)
        
#     return data_list

# # # Example usage
# # list_of_images_files = ["path/to/image1.tif", "path/to/image2.tif"]
# # label = "progressor"
# # data_dict = load_47_channel_tif_to_dataset_object(list_of_images_files, label)
# # print(data_dict)


In [ ]:
import numpy as np
from PIL import Image

def create_rgb_images_from_channels(channel_data, output_shape=(224, 224), values_per_pixel=3, output_path='output', filename_prefix='output'):
    """
    Create RGB images from channel data.

    Parameters:
        channel_data (list): List of channel arrays.
        output_shape (tuple): Desired shape of the output RGB images (default: (224, 224)).
        values_per_pixel (int): Number of values per pixel in the RGB images (default: 3).
        output_path (str): Path to save the RGB images (default: 'output').
        filename_prefix (str): Prefix for the output image filenames (default: 'output').

    Returns:
        list: List of RGB images.
    """
    num_channels = len(channel_data)
    values_per_rgb_image = np.prod(output_shape) * values_per_pixel
    total_values = num_channels * np.prod(channel_data[0].shape)
    num_images = total_values // values_per_rgb_image

    rgb_images = []

    for i in range(0, num_images * values_per_pixel, values_per_pixel):
        # Create an empty RGB image
        rgb_image = np.zeros((output_shape[0], output_shape[1], values_per_pixel), dtype=np.uint8)

        # Iterate through the next values_per_pixel channels
        for j, channel_index in enumerate(range(i, i + values_per_pixel)):
            if channel_index >= num_channels:
                break

            channel_values = channel_data[channel_index]  # Get channel data

            # Calculate the row and column index for the RGB image
            row_index = (j // values_per_pixel) * output_shape[0]
            col_index = (j % values_per_pixel) * output_shape[1]

            # Fill the RGB image with channel values
            rgb_image[row_index:row_index + output_shape[0], col_index:col_index + output_shape[1], j % values_per_pixel] = channel_values

        rgb_images.append(rgb_image)

    # Save the RGB images
    for i, rgb_image in enumerate(rgb_images):
        pil_image = Image.fromarray(rgb_image)
        file_name = f'{filename_prefix}_rgb_image_{i}.png'
        pil_image.save(f'{output_path}/{file_name}')

    return rgb_images


In [ ]:
# import numpy as np
# from PIL import Image

# def create_rgb_images_from_npz(npz_file_path, output_shape=(224, 224), values_per_pixel=3, output_path='output', filename_prefix='output'):
#     """
#     Create RGB images from channel data stored in an NPZ file.

#     Parameters:
#         npz_file_path (str): Path to the NPZ file containing channel data.
#         output_shape (tuple): Desired shape of the output RGB images (default: (224, 224)).
#         values_per_pixel (int): Number of values per pixel in the RGB images (default: 3).
#         output_path (str): Path to save the RGB images (default: 'output').
#         filename_prefix (str): Prefix for the output image filenames (default: 'output').

#     Returns:
#         list: List of RGB images.
#     """
#     # Load the NPZ file
#     npz_data = np.load(npz_file_path)

#     # Extract channel data
#     channel_data = [npz_data[f'arr_{i}'] for i in range(len(npz_data.files))]

#     return create_rgb_images_from_channels(channel_data, output_shape, values_per_pixel, output_path, filename_prefix)


In [ ]:
# def preprocess_image(example):
#     image = example['image']  # Assuming 'image' is the key in your dataset containing image data
#     inputs = feature_extractor(images=image, return_tensors="pt")
#     return inputs

# # preprocessed_dataset = dataset.map(preprocess_image)


In [ ]:
# import numpy as np
# from PIL import Image

# def create_rgb_images(raw_images, rgb_height, rgb_width):
#     num_channels, height, width = raw_images.shape
#     rgb_images = []

#     for y in range(0, height - rgb_height + 1):
#         rgb_image = np.zeros((rgb_height, rgb_width, 3), dtype=np.uint8)
#         for channel in range(num_channels):
#             channel_slice = raw_images[channel, y:y+rgb_height, 0:rgb_width]
#             rgb_image[:, :, channel] = channel_slice

#         rgb_images.append(rgb_image)

#     return rgb_images

# # # Example: Create a random 3x4x4 raw image
# # raw_images = np.random.randint(0, 256, size=(3, 4, 4), dtype=np.uint8)
# # print("Raw Images:\n", raw_images)

# # # Define the dimensions of the resulting RGB images
# # rgb_height = 2
# # rgb_width = 2

# # # Create RGB images using the function
# # rgb_images = create_rgb_images(raw_images, rgb_height, rgb_width)

# # # Print the created RGB images
# # for i, rgb_image in enumerate(rgb_images):
# #     print(f"RGB Image {i+1}:\n", rgb_image)


In [5]:
# num_channels = 42
# height = 1024
# width = 1024
# new_image_height = new_image_width = 224
# values_per_rgb_image = new_image_height * new_image_width * 3
# total_values = num_channels * height * width
# num_images = total_values // values_per_rgb_image
# num_images

292

In [7]:
# 224*224*3

150528

In [3]:
# channel1.shape[0] - 1 - 1  # Number of images we can create

2

In [8]:
# import numpy as np

# # Example input data for 3 channels
# channel1 = np.array([[1, 2, 3, 4],
#                      [5, 6, 7, 8],
#                      [9, 10, 11, 12],
#                      [13, 14, 15, 16]])

# channel2 = np.array([[17, 18, 19, 20],
#                      [21, 22, 23, 24],
#                      [25, 26, 27, 28],
#                      [29, 30, 31, 32]])

# channel3 = np.array([[33, 34, 35, 36],
#                      [37, 38, 39, 40],
#                      [41, 42, 43, 44],
#                      [45, 46, 47, 48]])

# # Combine the channels into a list
# channel_data = [channel1, channel2, channel3]

# # Create an empty array to store RGB images
# num_images = channel1.shape[0] - 1 - 1  # Number of images we can create
# rgb_images = np.zeros((num_images, 6, 6, 3), dtype=np.uint8)

# # Iterate through each image
# for img_index in range(num_images):
#     # Iterate through each channel
#     for channel_index in range(len(channel_data)):
#         # Extract 3 values from the current channel at the same position
#         values = channel_data[channel_index][img_index:img_index + 2, 0:2]

#         # Calculate the position in the RGB image
#         row_index = (channel_index // 3) // 2 * 2
#         col_index = (channel_index // 3) % 2 * 2

#         # Fill in the RGB image with the collapsed values
#         rgb_images[img_index, row_index:row_index + 2, col_index:col_index + 2, channel_index] = values

# print("RGB Images:")
# print(rgb_images)


RGB Images:
[[[[ 1 17 33]
   [ 2 18 34]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 5 21 37]
   [ 6 22 38]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]]


 [[[ 5 21 37]
   [ 6 22 38]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 9 25 41]
   [10 26 42]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]
   [ 0  0  0]]

  [[ 0  0  0]
   [ 0  0  0]
   [ 0  0  0